In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

#import beautifulsoup4
!conda install -c conda-forge beautifulsoup4 --yes
from bs4 import BeautifulSoup
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


#### get the Toronto neighborhood data from Wikipedia page

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url)
print(response.status_code)

200


### Get the table

In [3]:
page = response.text
html = BeautifulSoup(page)
table = html.find('table',"wikitable sortable")

### Get the header of the table
##### column name as columns

In [4]:
columns = []
for column in table.find_all('th'):
    columns.append(column.get_text().split('\n')[0])
print(columns)    

['Postal code', 'Borough', 'Neighborhood']


#### create the data frame table as df

In [5]:

df = pd.DataFrame([],columns = columns)

## the criteria
#### The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood  ##checked
#### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
#### More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
#### Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
#### In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

## My definition
#### create col_0,1,2 list to store the cell data and assign it in the df table

In [6]:

cols_0 = []
cols_1 = []
cols_2 = []
for row in table.find_all('tr')[1:]: # the real data is begin from 1
    col = row.find_all('td') #get the each row data
    if col[1].get_text() == 'Not assigned\n': # just store the assigned data
        continue
    
    col_0 = col[0].get_text().split('\n')[0] # clean the cell data, get rid of the \n in the cell data and add it in col list
    col_1 = col[1].get_text().split('\n')[0]
    if col[2].get_text() == '\n':
        col_2 = col[1].get_text().split('\n')[0] # if no neighborhood ,give the value of Borough
    else:
        col_2 = col[2].get_text().split('\n')[0].replace('/',',') #reshape the multi neighborhood 
    cols_0.append(col_0)
    cols_1.append(col_1)
    cols_2.append(col_2)

## Add the column data into the df table

In [7]:
df.iloc[:,0] = cols_0
df.iloc[:,1] = cols_1
df.iloc[:,2] = cols_2
df.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [8]:
df.shape

(103, 3)